<a href="https://colab.research.google.com/github/Kabindra02/Flood_Severity_index/blob/main/tcn_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U keras-tuner
!pip install keras==2.12.0
!pip install keras-tcn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-hub 0.21.1 requires keras>=3.5, but you have keras 2.12.0 which is incompatible.
tensorflow 2.19.0 requires keras>=3.5.0, but you have keras 2.12.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0


In [2]:
import pandas as pd
import numpy as np
import time
from collections import Counter

In [3]:
# Load your dataset

def load_dataset(path):
    return pd.read_csv(path)


file_path = "/content/df_with_flood_labels.csv"
df_flood= load_dataset(file_path)


df_flood

,Date,Daily flow Rate,Gauge height flow,24h accumulated,sm_top_0_10cm,sm_10_40cm,sm_change_rate,saturation_index,flow_change_rate,flow_lag_1d,...,Drainage_Density,Mean_CN,LULC_Forest_pct,LULC_Agriculture_pct,LULC_BuiltUp_pct,LULC_Grassland_pct,LULC_Water_pct,LULC_Barren_pct,flood_raw,flood_final
0,2009-01-03,456.0,3.33,0.0,22.784667,65.749364,4.666667,39.970546,-0.008696,460.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
1,2009-01-04,448.0,3.31,0.0,22.027667,68.656034,-0.757000,40.679014,-0.017544,456.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
2,2009-01-05,445.0,3.30,0.0,18.637667,58.216001,-3.390000,34.469000,-0.006696,448.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
3,2009-01-06,449.0,3.31,0.0,17.945667,56.251036,-0.692000,33.267815,0.008989,445.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
4,2009-01-07,453.0,3.32,0.0,18.687000,58.913774,0.741334,34.777710,0.008909,449.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5471,2023-12-27,342.0,3.11,0.0,15.803000,42.957146,-0.222999,26.664659,0.005882,340.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
5472,2023-12-28,343.0,3.11,0.0,15.623000,42.818542,-0.180000,26.501217,0.002924,342.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
5473,2023-12-29,343.0,3.11,0.0,15.421000,42.648949,-0.202001,26.312179,0.000000,343.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0
5474,2023-12-30,339.0,3.10,0.0,15.228000,42.470360,-0.193000,26.124944,-0.011662,343.0,...,0.000202,83.78663,0.4257,0.1268,0.1818,0.1567,0.0448,0.0001,0,0


In [6]:
df_flood['Date'] = pd.to_datetime(df_flood['Date'])
df_flood['day_of_year'] = df_flood['Date'].dt.dayofyear
df_flood['month'] = df_flood['Date'].dt.month
df_flood['season'] = (df_flood['Date'].dt.month % 12 + 3) // 3  # 1=Winter, 2=Spring, 3=Summer, 4=Fall
df_flood['day_of_week'] = df_flood['Date'].dt.dayofweek

In [11]:
features = ['Daily flow Rate', 'Gauge height flow', '24h accumulated', 'sm_top_0_10cm',
            'sm_10_40cm', 'sm_change_rate', 'saturation_index', 'flow_change_rate',
            'flow_lag_1d', 'flow_lag_2d', 'rain_sum_3d', 'rain_sum_5d',
            'rain_sum_7d', 'API', 'day_of_year', 'month', 'season', 'day_of_week']


In [12]:
df_input = df_flood[features]
df_target = df_flood['flood_final']

In [13]:
# Min-Max Scaling
def min_max_scaling(data):
    min_val = np.min(data)
    max_val = np.max(data)
    scaled_data = (data - min_val) / (max_val - min_val)
    return scaled_data, min_val, max_val

scaled_df = df_input.copy()
for column in features:
    scaled_df[column], _, _ = min_max_scaling(df_input[column])

print("Scaled features:")
print(scaled_df.head())

Scaled features:
   Daily flow Rate  Gauge height flow  24h accumulated  sm_top_0_10cm  \
0         0.014797           0.215627              0.0       0.420048   
1         0.014345           0.214125              0.0       0.396709   
2         0.014176           0.213373              0.0       0.292196   
3         0.014402           0.214125              0.0       0.270862   
4         0.014628           0.214876              0.0       0.293717   

   sm_10_40cm  sm_change_rate  saturation_index  flow_change_rate  \
0    0.397530        0.602599          0.404110          0.088179   
1    0.427545        0.377690          0.416303          0.086907   
2    0.319739        0.268505          0.309427          0.088467   
3    0.299448        0.380386          0.288754          0.090722   
4    0.326944        0.439823          0.314740          0.090710   

   flow_lag_1d  flow_lag_2d  rain_sum_3d  rain_sum_5d  rain_sum_7d  API  \
0     0.015023     0.015588          0.0          0.0 

In [14]:
# Prepare sequences for time series classification
def df_to_X_y_classification(df_features, df_target, window_size=5):
    df_as_np = df_features.to_numpy()
    target_as_np = df_target.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = df_as_np[i:i+window_size]
        X.append(row)
        label = target_as_np[i + window_size]
        y.append(label)
    return np.array(X), np.array(y)

In [15]:
WINDOW_SIZE = 2
X, y = df_to_X_y_classification(scaled_df, df_target, WINDOW_SIZE)

In [16]:
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Class distribution: {Counter(y)}")

X shape: (5474, 2, 18), y shape: (5474,)
Class distribution: Counter({np.int64(0): 5112, np.int64(2): 160, np.int64(1): 148, np.int64(3): 54})


In [17]:
#checking class imbalance
class_counts = Counter(y)
total_samples = len(y)
print("\nClass distribution:")
for cls, count in sorted(class_counts.items()):
    print(f"Class {cls}: {count} samples ({count/total_samples*100:.2f}%)")


Class distribution:
Class 0: 5112 samples (93.39%)
Class 1: 148 samples (2.70%)
Class 2: 160 samples (2.92%)
Class 3: 54 samples (0.99%)


In [18]:
!pip install hydroeval
!pip install keras-tcn

In [20]:
import hydroeval as he
from tcn import TCN
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop, SGD
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt
from tensorflow import keras
from sklearn.model_selection import KFold
from sklearn.metrics import (mean_squared_error, mean_absolute_error,
                            accuracy_score, precision_recall_fscore_support,
                            classification_report, confusion_matrix)
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from sklearn.utils.class_weight import compute_class_weight

In [21]:
# Compute class weights to handle imbalance
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)
class_weights = {i: class_weights_array[i] for i in range(len(class_weights_array))}
print(f"\nClass weights for imbalance handling: {class_weights}")



Class weights for imbalance handling: {0: np.float64(0.2677034428794992), 1: np.float64(9.246621621621621), 2: np.float64(8.553125), 3: np.float64(25.34259259259259)}


In [22]:
# Number of classes
num_classes = len(np.unique(y))
print(f"Number of classes: {num_classes}")

Number of classes: 4


In [26]:
# Define the TCN classification model
def create_tcn_classification_model(hp):
    input_layer = Input(shape=input_shape)

    # Number of TCN layers
    num_tcn_layers = hp.Int('num_tcn_layers', min_value=1, max_value=3, default=2)

    x = input_layer
    for i in range(num_tcn_layers):
        x = TCN(
            nb_filters=hp.Int(f'nb_filters_{i}', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice(f'kernel_size_{i}', values=[2, 3, 4]),
            dilations=[1, 2, 4, 8],
            dropout_rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1),
            return_sequences=True if i < num_tcn_layers - 1 else False
        )(x)

    # Add dropout before final layer
    x = Dropout(hp.Float('final_dropout', min_value=0.0, max_value=0.5, step=0.1))(x)

    # Output layer for classification with softmax activation
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    # Optimizer selection
    optimizer_name = hp.Choice('optimizer', values=['adam', 'adamax', 'rmsprop', 'sgd'])
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'adamax':
        optimizer = Adamax(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)

    # Use sparse categorical crossentropy (no need to one-hot encode y)
    # Note: Only using 'accuracy' during training to avoid batch size issues
    # Precision/Recall will be calculated after training using sklearn
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [27]:
# Define the input shape
input_shape = X.shape[1:]

In [28]:
# Initialize tracking variables
best_model_hyperparameters = None
best_model_accuracy = 0.0
best_model = None

worst_model_hyperparameters = None
worst_model_accuracy = 1.0
worst_model = None

In [ ]:
# Initialize K-Fold cross-validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Store results
nested_scores = []

for fold_outer_idx, (train_ix, test_ix) in enumerate(outer_cv.split(X)):
    print(f"\n{'='*60}")
    print(f"OUTER FOLD {fold_outer_idx + 1}/5")
    print(f"{'='*60}")

    X_train_outer, X_test_outer = X[train_ix], X[test_ix]
    y_train_outer, y_test_outer = y[train_ix], y[test_ix]

    # Check class distribution in training set
    print(f"Training set class distribution: {Counter(y_train_outer)}")
    print(f"Test set class distribution: {Counter(y_test_outer)}")

    # Inner loop for hyperparameter tuning
    inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

    for fold_idx, (train_ix_inner, val_ix) in enumerate(inner_cv.split(X_train_outer)):
        print(f"\n  Inner fold {fold_idx + 1}/3")

        X_train_inner, X_val = X_train_outer[train_ix_inner], X_train_outer[val_ix]
        y_train_inner, y_val = y_train_outer[train_ix_inner], y_train_outer[val_ix]

        # Initialize Keras Tuner
        tuner = RandomSearch(
            create_tcn_classification_model,
            objective='val_accuracy',
            max_trials=15,
            directory=f'/content/drive/MyDrive/research/tcn/keras_tuner_random_dir_fold_tcn__new_{fold_outer_idx}_{fold_idx}',
            project_name=f'/content/drive/MyDrive/research/tcn/keras_tuner_random_dir_fold_tcn_new_{fold_outer_idx}_{fold_idx}/hyperparameter_random_tuning_fold_tcn_{fold_outer_idx}_{fold_idx}'
        )

        # Search for best hyperparameters
        tuner.search(
            X_train_inner, y_train_inner,
            validation_data=(X_val, y_val),
            epochs=30,
            batch_size=32,
            class_weight=class_weights,
            verbose=0
        )

        best_hps = tuner.oracle.get_best_trials(1)[0].hyperparameters
        print(f"  Best hyperparameters: {best_hps.values}")

        # Train model with best hyperparameters
        model = create_tcn_classification_model(best_hps)
        es = EarlyStopping(patience=10, restore_best_weights=True, monitor='val_accuracy')

        history = model.fit(
            X_train_inner, y_train_inner,
            validation_data=(X_val, y_val),
            epochs=100,
            batch_size=32,
            callbacks=[es],
            class_weight=class_weights,
            verbose=0
        )

        # Evaluate on validation set
        y_pred_probs = model.predict(X_val, verbose=0)
        y_pred = np.argmax(y_pred_probs, axis=1)
        accuracy = accuracy_score(y_val, y_pred)
        print(f"  Validation accuracy: {accuracy:.4f}")

        # Update best model
        if accuracy > best_model_accuracy:
            best_model_accuracy = accuracy
            best_model_hyperparameters = best_hps
            best_model = model
            print(f"  New best model found!")

        # Update worst model
        if accuracy < worst_model_accuracy:
            worst_model_accuracy = accuracy
            worst_model_hyperparameters = best_hps
            worst_model = model

    # Evaluate best and worst models on test set
    print(f"\n{'='*60}")
    print(f"EVALUATING ON TEST SET (Outer Fold {fold_outer_idx + 1})")
    print(f"{'='*60}")

    # Best model evaluation
    start_time = time.time()
    y_pred_best_probs = best_model.predict(X_test_outer, verbose=0)
    end_time = time.time()
    y_pred_best = np.argmax(y_pred_best_probs, axis=1)
    time_duration = end_time - start_time

    accuracy_best = accuracy_score(y_test_outer, y_pred_best)
    precision_best, recall_best, f1_best, _ = precision_recall_fscore_support(
        y_test_outer, y_pred_best, average='weighted', zero_division=0
    )

    print("\nBest Model Results:")
    print(f"Accuracy: {accuracy_best:.4f}")
    print(f"Precision: {precision_best:.4f}")
    print(f"Recall: {recall_best:.4f}")
    print(f"F1-Score: {f1_best:.4f}")
    print(f"Testing time: {time_duration:.4f} seconds")
    print("\nClassification Report:")
    print(classification_report(y_test_outer, y_pred_best))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test_outer, y_pred_best))

    # Worst model evaluation
    y_pred_worst_probs = worst_model.predict(X_test_outer, verbose=0)
    y_pred_worst = np.argmax(y_pred_worst_probs, axis=1)

    accuracy_worst = accuracy_score(y_test_outer, y_pred_worst)
    precision_worst, recall_worst, f1_worst, _ = precision_recall_fscore_support(
        y_test_outer, y_pred_worst, average='weighted', zero_division=0
    )

    print("\nWorst Model Results:")
    print(f"Accuracy: {accuracy_worst:.4f}")
    print(f"Precision: {precision_worst:.4f}")
    print(f"Recall: {recall_worst:.4f}")
    print(f"F1-Score: {f1_worst:.4f}")

    nested_scores.append({
        "Best Model": {
            "Accuracy": accuracy_best,
            "Precision": precision_best,
            "Recall": recall_best,
            "F1-Score": f1_best,
            "Testing Time": time_duration,
            "Confusion Matrix": confusion_matrix(y_test_outer, y_pred_best).tolist(),
            "Predictions": y_pred_best.tolist()
        },
        "Worst Model": {
            "Accuracy": accuracy_worst,
            "Precision": precision_worst,
            "Recall": recall_worst,
            "F1-Score": f1_worst,
            "Confusion Matrix": confusion_matrix(y_test_outer, y_pred_worst).tolist(),
            "Predictions": y_pred_worst.tolist()
        }
    })


OUTER FOLD 1/5
Training set class distribution: Counter({np.int64(0): 4079, np.int64(2): 135, np.int64(1): 120, np.int64(3): 45})
Test set class distribution: Counter({np.int64(0): 1033, np.int64(1): 28, np.int64(2): 25, np.int64(3): 9})

  Inner fold 1/3
Reloading Tuner from /content/drive/MyDrive/research/tcn/keras_tuner_random_dir_fold_tcn_new_0_0/hyperparameter_random_tuning_fold_tcn_0_0/tuner0.json
  Best hyperparameters: {'num_tcn_layers': 2, 'nb_filters_0': 32, 'kernel_size_0': 3, 'dropout_0': 0.0, 'nb_filters_1': 96, 'kernel_size_1': 2, 'dropout_1': 0.0, 'final_dropout': 0.0, 'optimizer': 'adam', 'learning_rate': 0.0016978941148534812, 'nb_filters_2': 128, 'kernel_size_2': 2, 'dropout_2': 0.0}
  Validation accuracy: 0.8966
  New best model found!

  Inner fold 2/3
  Best hyperparameters: {'num_tcn_layers': 3, 'nb_filters_0': 96, 'kernel_size_0': 2, 'dropout_0': 0.0, 'nb_filters_1': 64, 'kernel_size_1': 2, 'dropout_1': 0.1, 'final_dropout': 0.1, 'optimizer': 'rmsprop', 'learnin

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Worst Model Results:
Accuracy: 0.8384
Precision: 0.9488
Recall: 0.8384
F1-Score: 0.8839

OUTER FOLD 2/5
Training set class distribution: Counter({np.int64(0): 4074, np.int64(2): 135, np.int64(1): 127, np.int64(3): 43})
Test set class distribution: Counter({np.int64(0): 1038, np.int64(2): 25, np.int64(1): 21, np.int64(3): 11})

  Inner fold 1/3
  Best hyperparameters: {'num_tcn_layers': 3, 'nb_filters_0': 32, 'kernel_size_0': 3, 'dropout_0': 0.0, 'nb_filters_1': 32, 'kernel_size_1': 4, 'dropout_1': 0.2, 'final_dropout': 0.30000000000000004, 'optimizer': 'adam', 'learning_rate': 0.003101886584898133, 'nb_filters_2': 32, 'kernel_size_2': 2, 'dropout_2': 0.0}
  Validation accuracy: 0.9144

  Inner fold 2/3
  Best hyperparameters: {'num_tcn_layers': 3, 'nb_filters_0': 128, 'kernel_size_0': 2, 'dropout_0': 0.2, 'nb_filters_1': 96, 'kernel_size_1': 4, 'dropout_1': 0.30000000000000004, 'final_dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.0017301949786254914, 'nb_filters_2': 32, 'kern

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Worst Model Results:
Accuracy: 0.8429
Precision: 0.9566
Recall: 0.8429
F1-Score: 0.8886

OUTER FOLD 3/5
Training set class distribution: Counter({np.int64(0): 4088, np.int64(2): 126, np.int64(1): 123, np.int64(3): 42})
Test set class distribution: Counter({np.int64(0): 1024, np.int64(2): 34, np.int64(1): 25, np.int64(3): 12})

  Inner fold 1/3
  Best hyperparameters: {'num_tcn_layers': 1, 'nb_filters_0': 32, 'kernel_size_0': 3, 'dropout_0': 0.0, 'nb_filters_1': 32, 'kernel_size_1': 4, 'dropout_1': 0.1, 'final_dropout': 0.30000000000000004, 'optimizer': 'adamax', 'learning_rate': 0.00013397773579515768}
  Validation accuracy: 0.8575

  Inner fold 2/3
  Best hyperparameters: {'num_tcn_layers': 3, 'nb_filters_0': 96, 'kernel_size_0': 4, 'dropout_0': 0.1, 'nb_filters_1': 64, 'kernel_size_1': 2, 'dropout_1': 0.1, 'final_dropout': 0.1, 'optimizer': 'sgd', 'learning_rate': 0.009270783532008702, 'nb_filters_2': 64, 'kernel_size_2': 2, 'dropout_2': 0.30000000000000004}
  Validation accuracy: 0

In [ ]:
print(f"\n{'='*60}")
print("FINAL SUMMARY")
print(f"{'='*60}")
print(f"\nBest Model Overall Accuracy: {best_model_accuracy:.4f}")
print(f"Best Hyperparameters: {best_model_hyperparameters.values}")
print(f"\nWorst Model Overall Accuracy: {worst_model_accuracy:.4f}")

In [ ]:
# Average metrics across all folds
avg_accuracy_best = np.mean([score["Best Model"]["Accuracy"] for score in nested_scores])
avg_precision_best = np.mean([score["Best Model"]["Precision"] for score in nested_scores])
avg_recall_best = np.mean([score["Best Model"]["Recall"] for score in nested_scores])
avg_f1_best = np.mean([score["Best Model"]["F1-Score"] for score in nested_scores])


In [ ]:
print(f"\nAverage Best Model Performance (across {len(nested_scores)} folds):")
print(f"Accuracy: {avg_accuracy_best:.4f} ± {np.std([s['Best Model']['Accuracy'] for s in nested_scores]):.4f}")
print(f"Precision: {avg_precision_best:.4f} ± {np.std([s['Best Model']['Precision'] for s in nested_scores]):.4f}")
print(f"Recall: {avg_recall_best:.4f} ± {np.std([s['Best Model']['Recall'] for s in nested_scores]):.4f}")
print(f"F1-Score: {avg_f1_best:.4f} ± {np.std([s['Best Model']['F1-Score'] for s in nested_scores]):.4f}")


In [ ]:
# Save the best model
best_model.save('best_tcn_flood_classification_model.h5')
print("\nBest model saved as 'best_tcn_flood_classification_model.h5'")